In [1]:
import pandas as pd
from gensim import corpora, models, similarities
import re

In [2]:
f = './actor_rpf_preprocessed.tsv'

In [3]:
df = pd.read_csv(f, sep = '\t')

In [6]:
lda = models.LdaModel.load('./actor_rpf')

In [15]:
def get_gamma(text, id2word):
    #pre-processing
    #input: a slice of the original df
    #output: a list of lists each containing cleaned words from a work
    text = text.lower().split()
    text = [re.sub(r'[^A-Za-z0-9]+', '', str(word)) for word in text]
    text = [word for word in text if len(word) > 1 and not word.isdigit()]
    gamma = lda.get_document_topics(id2word.doc2bow(text), minimum_probability = 0)
    return gamma

In [16]:
def preprocess(df):
    #pre-processing
    #input: a slice of the original df
    #output: a list of lists each containing cleaned words from a work
    df = df.astype(str)
    text = df[['Text']]
    text = text.drop_duplicates().Text.str.lower().str.split()
    text_cleaned = []
    for line in text:
        line = [re.sub(r'[^A-Za-z0-9]+', '', str(word)) for word in line]
        line = [word for word in line if len(word) > 1 and not word.isdigit()]
        text_cleaned.append(line)
    return text_cleaned

In [17]:
text_all = preprocess(df)
id2word = corpora.dictionary.Dictionary(text_all)

In [18]:
df['TopicDistribution'] = df.apply(lambda row:get_gamma(row['Text'], id2word), axis=1)

/home/yizhi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [21]:
df = df[['AdditionalTags', 'ArchiveWarnings', 'Author', 'Bookmarks',\
       'Category', 'ChapterIndex', 'Chapters', 'Characters', 'Comments',
       'CompleteDate', 'Fandoms', 'Hits', 'Kudos', \
       'PublishDate', 'Rating', 'Relationship', 'Title',
       'URL', 'UpdateDate', 'Words', 'TopicDistribution']]

In [ ]:
df.to_csv('shelock_processed_'+month, index = False)